In [1]:
import numpy as np
import numpy.polynomial.legendre as leg
import matplotlib.pyplot as plt
import scipy.special as sp
%matplotlib widget

plt.close('all')

# Obtain gauss-legendre quad points and eigenvalues
def analyze_quadrature(degree):
    print('\n Analysis for quad of degree' + str(degree))
    nodes, weights = leg.leggauss(degree)
    evs = np.arange(degree) + 0.5
    higher_degree = int(np.ceil(0.5 * (3.0 * degree - 1)))

    higher_nodes, higher_weights = leg.leggauss(higher_degree)

    # Build original basis Vandermonde matrix
    van = np.array([[leg.legval(nodes[i], (*np.zeros(j), 1))
                     for j in range(degree)]
                    for i in range(degree)])

    inv_van = np.array([[weights[j] * evs[i] * leg.legval(nodes[j], (*np.zeros(i), 1))
                         for j in range(degree)]
                        for i in range(degree)])

    # Function to test: elliptic cosine and its square
    k = 0.99 # elliptic modulus
    m = k ** 2.0 # elliptic parameter
    quarter_period = sp.ellipk(m)

    # Build grid: parameters
    elements = 10
    low, high = -quarter_period, quarter_period  # -np.pi, np.pi
    length = high - low
    dx = length / elements
    
    J = dx / 2.0

    # Build grid: construction
    def build_grid(order):
        n_o, w_o = leg.leggauss(order)
        nodes_iso = (np.array(n_o) + 1) / 2
        # element left boundaries (including ghost elements)
        xl = np.linspace(low, high-dx, num=elements)
        # construct coordinates
        grid_o = np.zeros((elements, order))
        for i in range(elements):
            grid_o[i, :] = xl[i] + dx * np.array(nodes_iso)
        return grid_o

    # Build grids
    grid, higher_grid = build_grid(degree), build_grid(higher_degree)
    cutoff = 20
    fundamental = 2.0 * np.pi / length
    wavenumbers = fundamental * np.arange(1-cutoff, cutoff)

    ### SINGLE QUAD
    # Build quad matrix for Fourier integral of an interpolant on the quad grid
    single_quad = np.multiply(weights[None, None, :], np.exp(-1j * np.tensordot(wavenumbers, grid, axes=0))) * J / length
    print(single_quad.shape)

    ### DOUBLE QUAD
    # Build quad matrix for Fourier integral of a product of interpolants on the quad grid
    inner_kernel = np.array([[[higher_weights[s] * 
                               leg.legval(higher_nodes[s], (*np.zeros(p), 1)) *
                               leg.legval(higher_nodes[s], (*np.zeros(q), 1))
                               for s in range(higher_degree)]
                              for p in range(degree)]
                             for q in range(degree)])
    modes_at_higher_degree = np.exp(-1j * np.tensordot(higher_grid, wavenumbers, axes=0))
    double_quad_inner = np.transpose(np.tensordot(inner_kernel, modes_at_higher_degree, axes=([2], [1])), (2, 0, 1, 3))
    # Do products with inverse vandermonde matrices
    double_quad = np.tensordot(inv_van,
                               np.tensordot(inv_van,
                                            double_quad_inner,
                                            axes=([0], [1])),
                               axes=([0], [2]))
    double_quad = np.transpose(double_quad, (3, 2, 0, 1)) * J / length
    print(double_quad.shape)

    ### TEST QUADS ON CN^2(x)
    # Test function: sin^2(x)
    f = sp.ellipj(grid.flatten(), m)[1].reshape(grid.shape)
    f_sq = sp.ellipj(grid.flatten(), m)[1].reshape(grid.shape) ** 2.0

    f_mult = np.einsum('mj,mk->mjk', f, f)

    def cn_sq_spectrum(n):
        if n == 0:
            return 1.0 + (sp.ellipe(m) / sp.ellipk(m) - 1.0) / m
        # if (n+1) % 2 == 0:
        #     return 0
        else:
            K = sp.ellipk(m)
            Kp = sp.ellipk(1-m)
            return np.pi**2.0 / (2.0 * m * K ** 2.0) * n / np.sinh(n * np.pi * Kp/K)

    spectrum_f = np.tensordot(single_quad, f_sq, axes=([1, 2], [0, 1]))
    spectrum_f_double = np.tensordot(double_quad, f_mult, axes=([1, 2, 3], [0, 1, 2]))
    

    waves = wavenumbers / fundamental

    true_spectrum = np.array([cn_sq_spectrum(waves[i]) for i in range(waves.shape[0])])

    fig, ax = plt.subplots(1, 2)
    ax[0].plot(grid.flatten(), f_sq.flatten(), 'o--'), ax[0].grid(True)
    ax[1].semilogy(waves, np.absolute(np.real(spectrum_f)), 'o', label='Single-quad')  # , ax[1].plot(wavenumbers, np.imag(spectrum_f), 'o')
    ax[1].semilogy(waves, np.absolute(np.real(spectrum_f_double)), 'o', label='Double-quad')  # , ax[1].plot(wavenumbers, np.imag(spectrum_f_double), 'o')
    ax[1].semilogy(waves, true_spectrum, 'o', label='True')
    ax[1].legend(loc='best'), ax[1].grid(True)

    error_single = np.absolute(np.real(spectrum_f) - true_spectrum)
    error_double = np.absolute(np.real(spectrum_f_double) - true_spectrum)

    middle_index = int((2*cutoff ) / 2)
    print(error_single[middle_index])
    print(error_double[middle_index])

    plt.figure()
    plt.semilogy(waves, error_single, 'o', label='single error')
    plt.semilogy(waves, error_double, 'o', label='double error')
    plt.ylabel(r'$|c - \widetilde{c}|$')
    plt.legend(loc='best'), plt.grid(True)

# Analysis of quadratures of various degrees:
### Note that order 1 results are identical in both respects (usual DFT)

In [2]:
analyze_quadrature(1)


 Analysis for quad of degree1
(39, 10, 1)
(39, 10, 1, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.4352936900274837e-05
1.4352936900274837e-05


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
analyze_quadrature(2)


 Analysis for quad of degree2
(39, 10, 2)
(39, 10, 2, 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3.4535749336706e-06
3.740896384399339e-05


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
analyze_quadrature(3)


 Analysis for quad of degree3
(39, 10, 3)
(39, 10, 3, 3)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

3.2227786958549665e-07
8.234915544746979e-07


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
analyze_quadrature(4)


 Analysis for quad of degree4
(39, 10, 4)
(39, 10, 4, 4)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.5325134911092775e-08
9.538201228842524e-09


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
analyze_quadrature(5)


 Analysis for quad of degree5
(39, 10, 5)
(39, 10, 5, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4.4192619008676104e-10
4.759613814186991e-10


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
analyze_quadrature(6)


 Analysis for quad of degree6
(39, 10, 6)
(39, 10, 6, 6)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

8.552963892682897e-12
7.586126171688079e-12


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
analyze_quadrature(7)


 Analysis for quad of degree7
(39, 10, 7)
(39, 10, 7, 7)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.1840528557627295e-13
1.1715628467356964e-13


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
analyze_quadrature(8)


 Analysis for quad of degree8
(39, 10, 8)
(39, 10, 8, 8)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.1102230246251565e-15
1.1657341758564144e-15


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
analyze_quadrature(9)


 Analysis for quad of degree9
(39, 10, 9)
(39, 10, 9, 9)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0
1.6653345369377348e-16


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
analyze_quadrature(10)


 Analysis for quad of degree10
(39, 10, 10)
(39, 10, 10, 10)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2.7755575615628914e-17
1.3877787807814457e-16


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
analyze_quadrature(12)


 Analysis for quad of degree12
(39, 10, 12)
(39, 10, 12, 12)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.1102230246251565e-16
5.551115123125783e-17


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### A quadrature of the Fourier integral to order (dx ^ n) in the product of interpolants improves the aliasing error